In [5]:
import pandas as pd

df = pd.read_csv('/Users/rohanmuthyala/Desktop/Airbnb-data/cleaned_airbnb_data.csv')

In [7]:
import sqlite3
conn = sqlite3.connect(':memory:')
df.to_sql('airbnb', conn, index=False, if_exists='replace')

52852

In [9]:
#Total and Average Revenue per Neighborhoods

In [11]:
query = '''
SELECT * FROM airbnb;
'''
result_df = pd.read_sql_query(query, conn)

print(result_df.head())

        id                                       name      host_id  \
0  1004098  Large Cozy 1 BR Apartment In Midtown East  45498551794   
1  1007411           Beautiful 1br on Upper West Side  18824631834   
2  1024533        Beautiful Sunny Park Slope Brooklyn  50357575975   
3  1025637                          West Side Retreat  55430108992   
4  1074240         Stylish  Sleek Apartment Near SoHo  24559680874   

  host_identity_verified host_name neighbourhood_group    neighbourhood  \
0               verified  Michelle           Manhattan      Murray Hill   
1               verified      Alan           Manhattan  Upper West Side   
2               verified     Alina            Brooklyn      South Slope   
3            unconfirmed    Alford           Manhattan  Upper West Side   
4               verified    Brooke           Manhattan     East Village   

   latitude  longitude        country  ... construction_year  price  \
0  40.74767  -73.97500  United States  ...              2

In [17]:
query = '''
SELECT neighbourhood_group, 
       COUNT(*) AS total_listings, 
       SUM(price) AS total_revenue,
       ROUND(AVG(price), 2) AS avg_price
FROM airbnb
GROUP BY neighbourhood_group
ORDER BY total_revenue DESC;
'''
result_df = pd.read_sql_query(query, conn)

print(result_df)


  neighbourhood_group  total_listings  total_revenue  avg_price
0            Brooklyn           23291     14608594.0     627.22
1           Manhattan           21180     13254705.0     625.81
2              Queens            6485      4060181.0     626.09
3               Bronx            1410       881377.0     625.09
4       Staten Island             486       296213.0     609.49


In [19]:
#Revenue Per Room Type in Each Neighborhood

In [26]:
query = '''
SELECT neighbourhood_group, room_type, 
       COUNT(*) AS total_listings, 
       SUM(price) AS total_revenue,
       ROUND(AVG(price), 2) AS avg_price
FROM airbnb
GROUP BY neighbourhood_group, room_type
ORDER BY total_revenue DESC;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)


   neighbourhood_group        room_type  total_listings  total_revenue  \
0            Manhattan  Entire home/apt           12462      7794345.0   
1             Brooklyn  Entire home/apt           12273      7648513.0   
2             Brooklyn     Private room           10763      6794784.0   
3            Manhattan     Private room            8339      5208333.0   
4               Queens     Private room            3383      2150106.0   
5               Queens  Entire home/apt            2972      1832803.0   
6                Bronx     Private room             801       503074.0   
7                Bronx  Entire home/apt             562       351010.0   
8            Manhattan      Shared room             365       243597.0   
9        Staten Island  Entire home/apt             273       172210.0   
10            Brooklyn      Shared room             253       164222.0   
11       Staten Island     Private room             212       123207.0   
12              Queens      Shared roo

In [19]:
#Occupancy Rate by Neighborhood Group

In [28]:
query = '''
SELECT neighbourhood_group,
       SUM(CASE WHEN availability_365 < 365 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS occupancy_rate
FROM airbnb
GROUP BY neighbourhood_group
ORDER BY occupancy_rate DESC;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)


  neighbourhood_group  occupancy_rate
0       Staten Island       96.502058
1            Brooklyn       96.088618
2           Manhattan       95.835694
3              Queens       95.512722
4               Bronx       94.326241


In [30]:
#Average Availability of Listings by Room Type

In [32]:
query = '''
SELECT room_type, AVG(availability_365) AS avg_availability
FROM airbnb
GROUP BY room_type
ORDER BY avg_availability;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)

         room_type  avg_availability
0       Hotel room         80.411765
1     Private room        115.943782
2  Entire home/apt        120.114673
3      Shared room        142.418868


In [34]:
#Average Review Scores by Neighborhood Group

In [41]:
query = '''
SELECT neighbourhood_group,
       ROUND(AVG(review_rate_number), 2) AS avg_review_score
FROM airbnb
GROUP BY neighbourhood_group
ORDER BY avg_review_score DESC;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)


  neighbourhood_group  avg_review_score
0               Bronx              3.37
1       Staten Island              3.33
2              Queens              3.30
3           Manhattan              3.24
4            Brooklyn              3.24


In [43]:
#Listings Distribution by Room Type

In [45]:
query = '''
SELECT room_type, COUNT(*) AS total_listings
FROM airbnb
GROUP BY room_type
ORDER BY total_listings DESC;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)

         room_type  total_listings
0  Entire home/apt           28542
1     Private room           23498
2      Shared room             795
3       Hotel room              17


In [47]:
#Hosts with the Highest Average Review Scores

In [49]:
query = '''
SELECT host_id, host_name, AVG(review_rate_number) AS avg_review_score
FROM airbnb
GROUP BY host_id, host_name
ORDER BY avg_review_score DESC
LIMIT 10;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)

     host_id                       host_name  avg_review_score
0  142125177                           Riley               5.0
1  148467235                       Stephanie               5.0
2  151277485                         Tatyana               5.0
3  158388023                         Maurice               5.0
4  162511322                            Hans               5.0
5  169042877                          Johnny               5.0
6  170610620                John And Colleen               5.0
7  209566505  Walker Hotel Greenwich Village               5.0
8  215491272                         Charlie               5.0
9  220979350                             Tov               5.0


In [51]:
#Correlation Between Price and Review Score

In [55]:
query = '''
SELECT room_type, 
       AVG(price) AS avg_price, 
       ROUND(AVG(review_rate_number),2) AS avg_review_score
FROM airbnb
GROUP BY room_type
ORDER BY avg_price DESC;
'''

result_df = pd.read_sql_query(query, conn)
print(result_df)


         room_type   avg_price  avg_review_score
0      Shared room  645.047799              3.20
1     Private room  628.968593              3.26
2  Entire home/apt  623.603146              3.25
3       Hotel room  580.705882              3.65


In [57]:
#Generate Summary Reports for Key Metrics

In [59]:
query = '''
SELECT neighbourhood_group, 
       COUNT(*) AS total_listings, 
       AVG(price) AS avg_price, 
       SUM(price) AS total_revenue,
       AVG(review_rate_number) AS avg_review_score
FROM airbnb
GROUP BY neighbourhood_group
ORDER BY total_revenue DESC;

'''

result_df = pd.read_sql_query(query, conn)
print(result_df)


  neighbourhood_group  total_listings   avg_price  total_revenue  \
0            Brooklyn           23291  627.220557     14608594.0   
1           Manhattan           21180  625.812323     13254705.0   
2              Queens            6485  626.088049      4060181.0   
3               Bronx            1410  625.090071       881377.0   
4       Staten Island             486  609.491770       296213.0   

   avg_review_score  
0          3.243656  
1          3.241313  
2          3.303470  
3          3.372340  
4          3.325103  
